In [2]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.technical import ma
import yfinance as yf
from pandas.tseries.offsets import BMonthEnd
import math
import alpaca_trade_api as tradeapi


In [14]:
data = yf.download("SPY", start="2000-02-29", end="2020-10-04")
data.to_csv("spy.csv")

[*********************100%***********************]  1 of 1 completed


In [15]:
class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        super(MyStrategy, self).__init__(feed)
        self.instrument = instrument
        self.position=None

        
    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        #self.info(f"BUY {execInfo.getQuantity()} shares at ${execInfo.getPrice()})")

    def onEnterCanceled(self, position):
        self.position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        #self.info(f"SELL {execInfo.getQuantity()} shares at ${execInfo.getPrice()})")
        self.position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.position.exitMarket()
    
    def onBars(self, bars):
        CLOSE = bars.getBar(self.instrument).getClose()
        DATE = bars.getBar(self.instrument).getDateTime()

        BROKER = self.getBroker()
        EQUITY = BROKER.getEquity()
        
        if(self.position is None):
            quantity = int(((self.getBroker().getCash()/CLOSE)*0.95))
            print(f"Buy on {DATE.date()}: {quantity} shares at {CLOSE}")
            self.position = self.enterLong(self.instrument, quantity, goodTillCanceled=True, allOrNone=False)
        
        if(self.position is not None):
            print(self.position.getReturn()*100)
            
        

# Load the bar feed from the CSV file
feed = yahoofeed.Feed()
feed.addBarsFromCSV("spy", "spy.csv")

# Evaluate the strategy with the feed's bars.
myStrategyInstance = MyStrategy(feed, "spy")
myStrategyInstance.run()




Buy on 2000-02-29: 6912 shares at 137.4375
0
0.5903723887375113
0.6584922797456858
2.5431425976385107
1.544050862851953
-0.4200726612170753
-0.544959128065395
2.361489554950045
1.8165304268846505
0.703905540417802
-0.7266121707538601
1.589464123524069
6.335149863760218
6.766575840145323
6.2216167120799275
8.401453224341509
9.059945504087194
10.92188919164396
11.580381471389646
10.399636693914623
9.763851044504996
9.877384196185286
8.038147138964577
9.264305177111716
9.900090826521344
9.08265213442325
8.401453224341509
9.34377838328792
10.036330608537693
9.60490463215259
9.287011807447774
6.289736603088102
4.813805631244324
-1.1807447774750226
2.2706630336058127
4.9727520435967305
3.9963669391462306
4.4959128065395095
3.3605812897366025
7.65213442325159
6.43732970027248
6.085376930063579
5.426884650317893
6.857402361489555
4.72297910990009
2.2706630336058127
3.0426884650317896
4.291553133514987
3.5081743869209805
2.679382379654859
0.36330608537693004
2.656675749318801
3.7693006357856493